# 16. Identify possible map coverage errors (required) (sequence diagram not required)
- Identify the geographic area defined as the difference between the geographic area covered by the
state and the geographic area covered by the union of all precincts in the state. 
- Store the associated data in the errors data structure for subsequent addition to the DB. 
- The regions so identified are potential ghost precincts. 

In [1]:
!ls

County                    RI_Geographic_Clean.ipynb State_Cleaned.json
County_Cleaned.json       RI_Prec_State_Union.json  test_ghost.json
DetectGhost.ipynb         RI_Precinct_Cleaned.json
Precinct                  State


In [2]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

In [3]:
precinctGDF = gpd.read_file("RI_Precinct_Cleaned.json")
precinctGDF = precinctGDF.to_crs("EPSG:3857").unary_union
#precinctGDF = precinctGDF.buffer(60.96)

In [4]:
gdf = gpd.GeoDataFrame({"geometry":precinctGDF})

In [5]:
stateGDF = gpd.read_file("State_Cleaned.json")
stateGDF = stateGDF.to_crs("EPSG:3857")

In [6]:
dif = gpd.overlay(stateGDF, gdf, how="difference")
difcopy = dif.copy()

In [7]:
# dif.to_file("RI_Ghost.json", driver="GeoJSON")

In [8]:
dif = difcopy.copy()
# dif = dif.to_crs("EPSG:3857")

In [9]:
a = 3716.1216
dif["filtered"] = ""
for index, row in dif.iterrows():
    if row.geometry.geom_type == "MultiPolygon":
        m = MultiPolygon([shape for shape in row["geometry"] if shape.area >= a])
        if m.is_empty:
            dif.drop(index, inplace=True)
        else:
            dif.at[index, "filtered"] = m
    else:
        if row["geometry"].area < a:
            dif.drop(index, inplace=True)
        else:
            dif.at[index, "filtered"] = row["geometry"]

In [10]:
dif.set_geometry("filtered", drop=True, inplace=True)

In [11]:
dif = dif.to_crs("EPSG:4326")

# turn all multipolygon into polygon

In [12]:
newgdf = gpd.GeoDataFrame(crs="EPSG:4326")
newgdf["geometry"] = ""
newgdf["index"] = 0

In [13]:
i = 0
for index, row in dif.iterrows():
    g = row["geometry"]
    if g.geom_type == "MultiPolygon":
        for shape in g:
            newgdf.at[i, "geometry"] = shape
            newgdf.at[i, "index"] = i
            i += 1
    else:
        newgdf.at[i, "geometry"] = g
        newgdf.at[i, "index"] = i
        i += 1
newgdf.set_geometry("geometry", inplace=True)

In [14]:
newgdf.head(1)

,geometry,index
0,"POLYGON ((-71.52526 41.38229, -71.52519 41.382...",0.0


In [15]:
#newgdf.to_file("test_ghost.json", driver="GeoJSON")

In [16]:
# dif.plot(figsize=(18, 18))

# delete unwanted polygons

In [17]:
def deletePoly(gdf, index):
    gdf.drop(index=index, inplace=True)
    return gdf

In [18]:
def filterByArea(gdf):
    gdf = gdf.to_crs("EPSG:3857")
    a = 3716.1216
    gdf["filtered"] = ""
    for index, row in gdf.iterrows():
        if row.geometry.geom_type == "MultiPolygon":
            m = MultiPolygon([shape for shape in row["geometry"] if shape.area >= a])
            if m.is_empty:
                gdf.drop(index, inplace=True)
            else:
                gdf.at[index, "filtered"] = m
        else:
            if row["geometry"].area < a:
                gdf.drop(index, inplace=True)
            else:
                gdf.at[index, "filtered"] = row["geometry"]
    gdf.set_geometry("filtered", drop=True, inplace=True)
    gdf = gdf.to_crs("EPSG:4326")
    return gdf

In [19]:
newgdf = filterByArea(newgdf)
newgdf.to_file("test_ghost.json", driver="GeoJSON")

In [20]:
index_to_delete = [46, 43, 44, 45, 39, 34, 33, 27, 26, 23, 14, 35, 38, 40, 42]
for i in index_to_delete:
    newgdf = deletePoly(newgdf, i)

# create key for ghost table

In [22]:
newgdf.head(1)

,geometry,index
0,"POLYGON ((-71.52526 41.38229, -71.52519 41.382...",0.0


In [23]:
newgdf["id"] = ""
i = 0
for index, row in newgdf.iterrows():
    newgdf.at[index, "id"] = "va-ghost-" + str(i)
    i += 1
newgdf.head(2)

,geometry,index,id
0,"POLYGON ((-71.52526 41.38229, -71.52519 41.382...",0.0,va-ghost-0
1,"POLYGON ((-71.76999 41.33280, -71.76997 41.332...",1.0,va-ghost-1


In [24]:
newgdf.to_file("RI_Ghost.json", driver="GeoJSON")